In [1]:
import numpy as np

from selenium import webdriver
import requests
from bs4 import BeautifulSoup
import pickle

import psycopg2

from datetime import datetime
import time

import os

In [2]:
os.chdir('Data')

In [3]:
params = {
    'host': 'localhost',
    'user': 'keith',
    'password': 'Password1@'
}

conn = psycopg2.connect(**params, dbname='jobs_links')
cur = conn.cursor()

In [4]:
def get_soup(url):
    chrome_options = webdriver.chrome.options.Options()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(executable_path='../../../anaconda3/bin/chromedriver',
                              options=chrome_options)
    driver.get(url)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    driver.close()
    return soup

To test the function you can run:

``` python
soup = get_soup('https://www.indeed.com/jobs?q=data+scientist&l=New+York')
```

Locations and key words on which to search

In [5]:
queries = [['Data', 'Scientist'], ['Machine', 'Learning', 'Engineer'],
           ['Data', 'Engineer'], ['Research', 'Engineer'],
           ['Artificial', 'Intelligence', 'Engineer'],
           ['Product', 'Analyst'], ['Product', 'Scientist']]

locations = [['Seattle'], ['New', 'York,+NY&_ga='],
             ['Los', 'Angeles'], ['Chicago'],
             ['San', 'Francisco', 'Bay', 'Area%2C+CA']]

Indeed will only return the first 1000 results when.  
Some queries have more jobs but the same jobs will continue to be returned after 1000.

Creates list of urls that will be scraped to get actual job URLs.

In [6]:
def get_job_listings_pages(locations, queries, cur):
    """
    Given a list of locations and job titles, runs a query for each job title
    for each location.
    Then calls the get_soup() function for each location/job combination.
    Grabs the number of results. On Indeed.com, if you specify page results higher
    than 1000, it will continue to return page results for 1000, therefore, if the
    reults are higher than 1000, it is capped.

    Results are then pickled.
    """
    
    date = datetime.strftime(datetime.today(), "%Y-%m-%d")

    for location in locations:

        urls = []

        for query in queries:

            time.sleep(np.random.poisson(100)/50)

            s_query = '+'.join(query)
            s_location = '+'.join(location)
            soup_url = ('https://www.indeed.com/jobs?q={}&l={}&start={}'
                        .format(s_query, s_location, 0))

            soup = get_soup(soup_url)

            num_postings_str = (soup.find(name='div', attrs={'id': "searchCount"})
                                .get_text().split()[-2])
            num_postings = int(num_postings_str.replace(',', ''))

            if num_postings > 1000:
                num_postings = 1000

            for i in range(0,num_postings,10):
                s_query = '+'.join(query)
                s_location = '+'.join(location)
                base_url = 'https://www.indeed.com/jobs?q={}&l={}&start={}'.format(s_query, s_location, i)

                cur.execute("INSERT INTO job_listings_pages \
                            (date_pulled, query, city, link) VALUES ('"
                            + date + "', '" + s_query + "', '" + s_location + "', '" + base_url
                            + "');")
                conn.commit()

In [7]:
get_job_listings_pages(locations, queries, cur)

10 14 7 62
10 14 7 63
10 14 7 63
10 14 7 63
10 14 7 63
10 14 7 63
10 14 7 63
10 14 7 63
10 14 7 63
10 14 7 63
10 14 7 64
10 14 7 64
10 14 7 64
10 14 7 64
10 14 7 64
10 14 7 64
10 14 7 64
10 14 7 64
10 14 7 64
10 14 7 64
10 14 7 64
10 14 7 64
10 14 7 64
10 14 7 64
10 14 7 64
10 14 7 64
10 14 7 64
10 14 7 64
10 14 7 64
10 14 7 64
10 14 7 64
10 14 7 64
10 14 7 64
10 14 7 64
10 14 7 64
10 14 7 64
10 14 7 64
10 14 7 64
10 14 7 64
10 14 7 64
10 14 7 64
10 14 7 64
10 14 7 64
10 14 7 64
10 14 7 64
10 14 7 64
10 14 7 64
10 14 7 64
10 14 7 64
10 14 7 64
10 14 7 64
10 14 7 64
10 14 7 64
10 14 7 64
10 14 7 64
10 14 7 64
10 14 7 64
10 14 7 64
10 14 7 64
10 14 7 64
10 14 7 64
10 14 7 64
10 14 7 64
10 14 7 64
10 14 7 64
10 14 7 64
10 14 7 64
10 14 7 64
10 14 7 64
10 14 7 64
10 14 7 64
10 14 7 64
10 14 7 64
10 14 7 64
10 14 7 64
10 14 7 64
10 14 7 64
10 14 7 64
10 14 7 64
10 14 7 64
10 14 7 64
10 14 7 64
10 14 7 64
10 14 7 64
10 14 7 64
10 14 7 64
10 14 7 64
10 14 7 64
10 14 7 64
10 14 7 64
10 14 7 64

In [10]:
urls = ['www.test.com', 'www.test.com', 'www.test.com', 'www.test.com']
date = datetime.strftime(datetime.today(), "%Y-%m-%d")
query = 'test'
location = 'here'

In [11]:
[(date, location, query, url) for url in urls]

[('2019-07-01', 'here', 'test', 'www.test.com'),
 ('2019-07-01', 'here', 'test', 'www.test.com'),
 ('2019-07-01', 'here', 'test', 'www.test.com'),
 ('2019-07-01', 'here', 'test', 'www.test.com')]

In [8]:
date = datetime.strftime(datetime.today(), "%Y-%m-%d")

In [9]:
date

'2019-07-01'

In [10]:
len(date)

10